In [2]:
import pandas as pd
import numpy as np

In [3]:
msa_list = pd.read_csv("data/wikipedia_msa.csv", header=0, delimiter=",", encoding="utf-8")
zip_msa = pd.read_csv("data/zip07_cbsa06.txt", header=0, delimiter=",", encoding="latin1", low_memory=False)
#print(zip_msa[zip_msa['CBSA LSAD'] == 'Metropolitan Statistical Area']['CBSA TITLE'].sort_values().unique())

In [4]:
msa_list.loc[:, 'Metropolitan statistical area'] = [x.replace('?', '-').replace(' MSA', '') for x in msa_list.loc[:, 'Metropolitan statistical area']]

cities = ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Atlanta', 'Phoenix', 'Boston', 'San Francisco', 'Detroit', 'San Diego', 'Denver', 'Baltimore', 'Charlotte-', 'Orlando', 'San Antonio', 'Portland', 'Sacramento', 'Austin', 'Las Vegas', 'Cincinnati', 'Indianapolis', 'Cleveland', 'Nashville', 'Providence', 'Milwaukee', 'New Orleans', 'Hartford', 'Grand Rapids', 'Honolulu', 'Worcester', 'Greenville-', 'Sarasota', 'Boise', 'Lakeland', 'Durham', 'Spokane', 'Scranton']

zcities = [zip_msa[zip_msa['CBSA TITLE'].str.contains(city, na=False)]['CBSA TITLE'].iloc[0] for city in cities]
mcities = [msa_list[msa_list['Metropolitan statistical area'].str.contains(city, na=False)]['Metropolitan statistical area'].iloc[0] for city in cities]

for i in range(len(cities)):
    msa_list['Metropolitan statistical area'] = [x.replace(mcities[i], zcities[i]) for x in msa_list['Metropolitan statistical area']]


In [5]:
for i in range(5):
    print(zcities[i], ' | ', mcities[i])

New York-Northern New Jersey-Long Island, NY-NJ-PA  |  New York-Newark-Jersey City, NY-NJ-PA
Los Angeles-Long Beach-Santa Ana, CA  |  Los Angeles-Long Beach-Anaheim, CA
Chicago-Naperville-Joliet, IL-IN-WI  |  Chicago-Naperville-Elgin, IL-IN-WI
Houston-Sugar Land-Baytown, TX  |  Houston-The Woodlands-Sugar Land, TX
Atlanta-Sandy Springs-Marietta, GA  |  Atlanta-Sandy Springs-Alpharetta, GA


In [6]:
#print(msa_list[msa_list['Metropolitan statistical area'].str.contains('New York', na=False)]['Metropolitan statistical area'].iloc[0])
print(msa_list['Metropolitan statistical area'])

0      New York-Northern New Jersey-Long Island, NY-N...
1                   Los Angeles-Long Beach-Santa Ana, CA
2                    Chicago-Naperville-Joliet, IL-IN-WI
3                        Dallas-Fort Worth-Arlington, TX
4                         Houston-Sugar Land-Baytown, TX
                             ...                        
379                                         Danville, IL
380                                      Lewiston, ID-WA
381                                      Walla Walla, WA
382                                             Enid, OK
383                                      Carson City, NV
Name: Metropolitan statistical area, Length: 384, dtype: object


In [13]:

#print(zip_msa[zip_msa['CBSA TITLE'].str.contains('New York', na=False)].loc[:, ['CBSA TITLE'
#print(len(msa_lookup[~msa_lookup.isin(zip_msa['CBSA TITLE'])]))
#print(msa_lookup[~msa_lookup.isin(zip_msa['CBSA TITLE'])])
msa_lookup = msa_list.loc[msa_list['Rank'] <= 100, 'Metropolitan statistical area']

zip_msa1 = zip_msa[(zip_msa['CBSA TITLE'].isin(msa_lookup))]
zip_msa1 = zip_msa1.loc[:, ['ZIP5', 'STATE', 'CBSA TITLE']]
zip_msa1['CBSA TITLE'] = [x.replace('--', '-') for x in zip_msa1['CBSA TITLE']]
zip_msa1['ZIP5'] = [str(i) for i in zip_msa1['ZIP5']]
zip_msa1.loc[zip_msa1['ZIP5'].str.len() == 3, 'ZIP5'] = ['00' + i for i in zip_msa1.loc[zip_msa1['ZIP5'].str.len() == 3, 'ZIP5']]
zip_msa1.loc[zip_msa1['ZIP5'].str.len() == 4, 'ZIP5'] = ['0' + i for i in zip_msa1.loc[zip_msa1['ZIP5'].str.len() == 4, 'ZIP5']]

#print(zip_msa1['CBSA TITLE'].unique())
#print(len(zip_msa1['CBSA TITLE'].unique()))
#zip_msa1['ZIP5'].unique()

zip_lookup = zip_msa1.loc[:, 'ZIP5']

print(zip_msa1[zip_msa1['CBSA TITLE'].str.contains('Boston')])


          ZIP5 STATE                      CBSA TITLE
30530    01901    MA  Boston-Cambridge-Quincy, MA-NH
30531    01902    MA  Boston-Cambridge-Quincy, MA-NH
30532    01903    MA  Boston-Cambridge-Quincy, MA-NH
30533    01904    MA  Boston-Cambridge-Quincy, MA-NH
30534    01905    MA  Boston-Cambridge-Quincy, MA-NH
...        ...   ...                             ...
2713033  01464    MA  Boston-Cambridge-Quincy, MA-NH
2713036  01469    MA  Boston-Cambridge-Quincy, MA-NH
2713037  01471    MA  Boston-Cambridge-Quincy, MA-NH
2713038  01472    MA  Boston-Cambridge-Quincy, MA-NH
2713040  01474    MA  Boston-Cambridge-Quincy, MA-NH

[2472 rows x 3 columns]


In [18]:
rdc_core = pd.read_csv("data/RDC_Inventory_Core_Metrics_Zip_History.csv", header=0, delimiter=",", encoding="utf-8", low_memory=False)
rdc_hotness = pd.read_csv("data/RDC_Inventory_Hotness_Metrics_Zip_History.csv", header=0, delimiter=",", encoding="utf-8", low_memory=False)

In [19]:
#print(rdc_core.head(10))
print(rdc_core.shape)
print(rdc_core['month_date_yyyymm'].min())

rdc_core = rdc_core[(rdc_core['postal_code'] != ' please contact economics@realtor.com for more details.')]
rdc_core['month_date_yyyymm'] = rdc_core['month_date_yyyymm'].astype(int)

rdc_core = rdc_core[(rdc_core['month_date_yyyymm'] >= 201901) & (rdc_core['postal_code'].isin(zip_lookup))].drop([''])
print(rdc_core.shape)

(2099207, 40)
201607
(513358, 40)


In [25]:
rdc_hotness.head(10)
print(rdc_hotness.shape)
print(rdc_hotness['month_date_yyyymm'].min())
print(rdc_hotness['postal_code'].min())

rdc_hotness = rdc_hotness[(rdc_hotness['postal_code'] != ' please contact economics@realtor.com for more details.')]
rdc_hotness['month_date_yyyymm'] = rdc_hotness['month_date_yyyymm'].astype(int)
rdc_hotness = rdc_hotness[(rdc_hotness['month_date_yyyymm'] >= 201901) & (rdc_hotness['postal_code'].isin(zip_lookup))]
rdc_hotness['zip_name'].str.replace(r'\b[a-z](?=.*,)', lambda x: x.str.capitalize(), regex=True)
print(rdc_hotness.shape)
print(rdc_hotness['postal_code'].min())

(312905, 24)
201901
01001
(312905, 24)
01001


In [28]:
zip_msa1.to_csv('data/zip_msa1.csv', index=False, encoding='latin1')
rdc_hotness.to_csv('data/rdchotness.csv', index=False, encoding='utf-8')
rdc_core.to_csv('data/rdc_core.csv', index=False, encoding='utf-8')

In [29]:
rdc_hotness['zip_name'].str.replace(r'\b[a-z](?=.*,)', lambda x: x.str.capitalize(), regex=True)

0        NaN
1        NaN
5        NaN
6        NaN
7        NaN
          ..
787984   NaN
787985   NaN
787988   NaN
787991   NaN
787992   NaN
Name: zip_name, Length: 312905, dtype: float64